In [21]:
from coffea.nanoevents import TreeMakerSchema, BaseSchema, NanoEventsFactory
# import uproot as up
# import pandas as pd
# import numpy as np
import os
import awkward as ak
import mplhep as hep
hep.style.use(hep.style.CMS)
import matplotlib.pyplot as plt
import functools

from helpers import get_summary_of_cuts
%matplotlib agg

### Read Data and MC

In [22]:
fname_data = '../inputFiles/crab_TTree_13TeV_zeeymm_2022BCDE.root'
savepath = 'plots/optimization/data/'

# fname_mc = '../output/ZmmYee/MC/ZmmMCOut.root'
# savepath = 'plots/Optimization/MC/'

In [23]:
# make directory if it doesn't exist
if not os.path.exists(savepath):
    os.makedirs(savepath)

In [24]:
events_data = NanoEventsFactory.from_root({fname_data: "ntuple"},
     schemaclass=BaseSchema,
     entry_stop=100,
).events()

In [25]:
columns = ['Event', 'Run', 'LumiBlock',
           'B_Z_TriggerPath', 'B_Z_TriggerPath1',
           'B_Z_VtxMass', 'B_Z_mass',
           'B_Z_pt1', 'B_Z_pt2', 'B_Z_eta1', 'B_Z_eta2',
           'B_Z_mvaIsoWP90_1', 'B_Z_mvaIsoWP90_2',
           'B_Z_VtxProb', 

           'B_J_VtxMass', 'B_J_mass',
           'B_J_pt1', 'B_J_pt2', 'B_J_eta1', 'B_J_eta2',
           'B_J_soft1', 'B_J_soft2',
           'B_J_VtxProb',
           
           'FourL_mass', 'FourL_VtxProb']

events_data = events_data[columns].compute()

In [26]:
# convert float variables to boolean
events_data['B_Z_TriggerPath'] = events_data['B_Z_TriggerPath'] != 0
events_data['B_Z_TriggerPath1'] = events_data['B_Z_TriggerPath1'] != 0

## Define cut objects

In [27]:
def make_cut_list(events):
    cut_fake = (events['B_Z_mass'] > -1)

    cut_unOrdered_Z = events['B_Z_pt1'] > events['B_Z_pt2']
    cut_unOrdered_J = events['B_J_pt1'] > events['B_J_pt2']
    cut_unOrdered = cut_unOrdered_Z & cut_unOrdered_J

    cut_EleTrigger = events['B_Z_TriggerPath']
    cut_Jsoft = events['B_J_soft1'] & events['B_J_soft2']
    cut_EleTrigEnforce = events['B_Z_pt1'] > 30
    cut_dilepton_prob = (events['B_J_VtxProb'] > 0.01) & (events['B_Z_VtxProb'] > 0.01)
    cut_FourL_prob = events['FourL_VtxProb'] > 0.01

    cut_Pt = (events['B_Z_pt1'] > 27.0) & (events['B_Z_pt2'] > 5.0) & (events['B_J_pt1'] > 3.0) & (events['B_J_pt2'] > 3.0)
    cut_eta = (abs(events['B_Z_eta1']) < 2.4) & (abs(events['B_Z_eta2']) < 2.4) & (abs(events['B_J_eta1']) < 2.5) & (abs(events['B_J_eta2']) < 2.5)
    cut_detector = cut_Pt & cut_eta

    cut_Jmass = (events['B_J_mass'] > 9.2) & (events['B_J_mass'] < 9.7)
    cut_Zmass = (events['B_Z_mass'] > 70) & (events['B_Z_mass'] < 110)

    cut_ZmvaIsoHighpT = events['B_Z_mvaIsoWP90_1']
    cut_ZmvaIsoLowpT = events['B_Z_mvaIsoWP90_2']
    cut_ZmvaIsoBoth = (events['B_Z_mvaIsoWP90_1']) | (events['B_Z_mvaIsoWP90_2'])

    cut_FourLmass = (events['FourL_mass'] > 112) & (events['FourL_mass'] < 162)

    cut_list = {
        "0": {"name": "Preselection", "mask": cut_fake},
        "1": {"name": "UnOrdered pT", "mask": cut_unOrdered},
        "2": {"name": "Electron Trigger", "mask": cut_EleTrigger},
        "3": {"name": "Soft Muons", "mask": cut_Jsoft},
        "4": {"name": "Electron Trigger Enforce", "mask": cut_EleTrigEnforce},
        "5": {"name": "Dilepton Vtx > 1%", "mask": cut_dilepton_prob, "var": ["B_J_VtxProb", "B_Z_VtxProb"]},
        "6": {"name": "FourL Vtx > 1%", "mask": cut_FourL_prob, "var": ["FourL_VtxProb"]},
        "7": {"name": "Detector acceptance", "mask": cut_detector},
        "8": {"name": "J mass", "mask": cut_Jmass, "var": ["B_J_mass"]},
        "9": {"name": "Z mass", "mask": cut_Zmass, "var": ["B_Z_mass"]},
        "10": {"name": "eleID High pT", "mask": cut_ZmvaIsoHighpT, "var": ["B_J_pt1"]},
        "11": {"name": "eleID Low pT", "mask": cut_ZmvaIsoLowpT, "var": ["B_J_pt2"]},
        "12": {"name": "eleID either", "mask": cut_ZmvaIsoBoth, "var": ["B_J_pt1", "B_J_pt2"]},
        "13": {"name": "FourL mass", "mask": cut_FourLmass, "var": ["FourL_mass"]}
    }

    return cut_list

In [28]:
dilepton_mass_cuts = [8, 9]
eleID_cuts = [10, 11, 12]

cut_list_data = make_cut_list(events_data)

In [29]:
# eleID_highPt = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 13]
# eleID_lowPt = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 13]
# eleID_both = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 12, 13]
eleID_both = [0, 1, 2, 3, 12, 7, 5, 8, 9, 6, 13]

# eleID_highPt_noMass = [x for x in eleID_highPt if x not in dilepton_mass_cuts]
# eleID_lowPt_noMass = [x for x in eleID_lowPt if x not in dilepton_mass_cuts]
# eleID_both_noMass = [x for x in eleID_both if x not in dilepton_mass_cuts]

## Function Definitions

#### Functions for plots at specific cut point

In [30]:
def get_view_at(mycut, summary_dict, cut_list, events):
    mycut_name = cut_list[str(mycut)]['name']

    # find the cut in summary dict
    cut_index = summary_dict['Cut'].index(mycut_name)
    n_events_after_cut = summary_dict['Events'][cut_index]

    view_index = cut_index-1
    view_index_name = summary_dict['Cut'][view_index]
    view_index_mask = summary_dict['Aggregated mask'][view_index]
    n_events_before_cut = summary_dict['Events'][view_index]

    cut_of_interest = events[view_index_mask]
    text_array = [view_index_name, mycut_name, n_events_before_cut, n_events_after_cut]

    return cut_of_interest, text_array

def make_hist(nbins, xlow, xhigh, values, labels, lines, fileName, xlabel, text_array):
    cut_name_at_plot, cut_name_after_plot, n_ev_before, n_eve_after = text_array
    
    unit = 'GeV' if 'eta' not in xlabel else ''
    
    plt.figure(figsize=(8, 8))
    
    for i, value in enumerate(values):
        plt.hist(value, bins=nbins, range=(xlow, xhigh), label=labels[i], alpha=0.5)
    
    for line in lines:
        plt.axvline(x=line, color='r')

    plt.text(0.5, 0.5, f"Drawn at: {cut_name_at_plot} ({n_ev_before})", fontsize=12, transform=plt.gca().transAxes)
    plt.text(0.5, 0.45, f"Next cut: {cut_name_after_plot} ({n_eve_after})", fontsize=12, transform=plt.gca().transAxes)

    plt.xlabel(xlabel)
    plt.ylabel(f"Counts / {(xhigh-xlow)/nbins:.2f} {unit}")
    plt.legend(fontsize=13)
    plt.tight_layout()
    plt.savefig(f"{savepath}/{fileName}.png")

def make_hist2D(xvar, yvar, xlabel, ylabel, fileName, text_array):
    cut_name_at_plot, cut_name_after_plot, n_ev_before, n_eve_after = text_array
    
    plt.figure(figsize=(8, 8))

    plt.hist2d(xvar, yvar, bins=(50, 50), range=((-3, 3), (0, 50)))
    plt.colorbar()
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.tight_layout()
    plt.savefig(f"{savepath}/{fileName}.png")

In [31]:
def plot_dilepton_vertexing(cut_of_interest, text_array):
    J_vtx_prob = ak.flatten(cut_of_interest['B_J_VtxProb'])
    Z_vtx_prob = ak.flatten(cut_of_interest['B_Z_VtxProb'])

    nbins, xlow, xhigh = 100, 0, 1
    fileName = "vtx_prob_dilepton"
    values = [J_vtx_prob, Z_vtx_prob]
    labels = ["J", "Z"]
    lines = [0.01]
    xlabel = "Dilepton Vtx Prob"

    make_hist(nbins, xlow, xhigh, values, labels, lines, fileName, xlabel, text_array)

def plot_dielectron_inv_mass(cut_of_interest, text_array, isFitData=False):
    dielectron_mass = ak.flatten(cut_of_interest['B_Z_mass'])

    if isFitData:
        nameExt = 'fitData'
        lines = []
    else:
        nameExt = ''
        lines = [8, 10]

    nbins, xlow, xhigh = 20, 0, 12
    fileName = f"Z_mass {nameExt}"
    values = [dielectron_mass]
    labels = ["Z_mass"]
    xlabel = "Dielectron inv mass [GeV]"

    make_hist(nbins, xlow, xhigh, values, labels, lines, fileName, xlabel, text_array)

def plot_dimuon_inv_mass(cut_of_interest, text_array, isFitData=False):
    dimuon_mass = ak.flatten(cut_of_interest['B_J_mass'])

    if isFitData:
        nameExt = 'fitData'
        lines = []
    else:
        nameExt = '' 
        lines = [70, 110]

    nbins, xlow, xhigh = 20, 60, 120
    fileName = f"J_mass {nameExt}"
    values = [dimuon_mass]
    labels = ["J_mass"]
    xlabel = "Dimuon inv mass [GeV]"

    make_hist(nbins, xlow, xhigh, values, labels, lines, fileName, xlabel, text_array)

def plot_fourL_vertexing(cut_of_interest, text_array):
    FourL_vtx_prob = ak.flatten(cut_of_interest['FourL_VtxProb'])

    nbins, xlow, xhigh = 100, 0, 1
    fileName = "vtx_prob_fourL"
    values = [FourL_vtx_prob]
    labels = ["FourL"]
    lines = [0.01]
    xlabel = "FourL Vtx Prob"

    make_hist(nbins, xlow, xhigh, values, labels, lines, fileName, xlabel, text_array)

def plot_muon_pt(cut_of_interest, text_array):
    muon_pt1 = ak.flatten(cut_of_interest['B_J_pt1'])
    muon_pt2 = ak.flatten(cut_of_interest['B_J_pt2'])

    nbins, xlow, xhigh = 75, 0, 20
    fileName = "Mu_pt"
    values = [muon_pt1, muon_pt2]
    labels = ["Mu_pt1", "Mu_pt2"]
    lines = [3.0]
    xlabel = "Mu pT [GeV]"

    make_hist(nbins, xlow, xhigh, values, labels, lines, fileName, xlabel, text_array)

def plot_electron_pt(cut_of_interest, text_array):
    ele_pt1 = ak.flatten(cut_of_interest['B_Z_pt1'])
    ele_pt2 = ak.flatten(cut_of_interest['B_Z_pt2'])

    nbins, xlow, xhigh = 50, 0, 50
    fileName = "Ele_pt"
    values = [ele_pt1, ele_pt2]
    labels = ["Ele_pt1", "Ele_pt2"]
    lines = [5.0]
    xlabel = "Ele pT [GeV]"

    make_hist(nbins, xlow, xhigh, values, labels, lines, fileName, xlabel, text_array)

def plot_muon_eta(cut_of_interest, text_array):
    muon_eta1 = ak.flatten(cut_of_interest['B_J_eta1'])
    muon_eta2 = ak.flatten(cut_of_interest['B_J_eta2'])

    nbins, xlow, xhigh = 30, -3, 3
    fileName = "Mu_eta"
    values = [muon_eta1, muon_eta2]
    labels = ["Mu_eta1", "Mu_eta2"]
    lines = [2.4]
    xlabel = "Mu eta"

    make_hist(nbins, xlow, xhigh, values, labels, lines, fileName, xlabel, text_array)

def plot_electron_eta(cut_of_interest, text_array):
    ele_eta1 = ak.flatten(cut_of_interest['B_Z_eta1'])
    ele_eta2 = ak.flatten(cut_of_interest['B_Z_eta2'])

    nbins, xlow, xhigh = 30, -3, 3
    fileName = "Ele_eta"
    values = [ele_eta1, ele_eta2]
    labels = ["Ele_eta1", "Ele_eta2"]
    lines = [2.5]
    xlabel = "Ele eta"

    make_hist(nbins, xlow, xhigh, values, labels, lines, fileName, xlabel, text_array)

def plot_electron_pt_vs_eta(cut_of_interest, text_array):
    xvar = ak.flatten(cut_of_interest['B_Z_eta1']).tolist()
    yvar = ak.flatten(cut_of_interest['B_Z_pt1']).tolist()

    fileName = "Ele1_eta_pt"
    xlabel = "Ele eta1"
    ylabel = "Ele pT1"
    make_hist2D(xvar, yvar, xlabel, ylabel, fileName, text_array)

    xvar = ak.flatten(cut_of_interest['B_Z_eta2']).tolist()
    yvar = ak.flatten(cut_of_interest['B_Z_pt2']).tolist()

    fileName = "Ele2_eta_pt"
    xlabel = "Ele eta2"
    ylabel = "Ele pT2"
    make_hist2D(xvar, yvar, xlabel, ylabel, fileName, text_array)

def plot_fourL_inv_mass(cut_of_interest, text_array):
    fourL_mass = ak.flatten(cut_of_interest['FourL_mass'])

    nameExt = text_array[0] if 'eleID' in text_array[0] else ''

    nbins, xlow, xhigh = 10, 70, 170
    fileName = f"FourL_mass_with {nameExt} cut"
    values = [fourL_mass]
    labels = ["FourL_mass"]
    lines = [112, 162]
    xlabel = "FourL inv mass [GeV]"

    make_hist(nbins, xlow, xhigh, values, labels, lines, fileName, xlabel, text_array)

In [32]:
def show_plots_at_each_cut(summary_dict, cut_list, events):
    cut_of_interest, text_array = get_view_at(5, summary_dict, cut_list, events)
    plot_dilepton_vertexing(cut_of_interest, text_array)

    cut_of_interest, text_array = get_view_at(6, summary_dict, cut_list, events)
    plot_fourL_vertexing(cut_of_interest, text_array)

    cut_of_interest, text_array = get_view_at(7, summary_dict, cut_list, events)
    plot_muon_pt(cut_of_interest, text_array)
    plot_electron_pt(cut_of_interest, text_array)
    plot_muon_eta(cut_of_interest, text_array)
    plot_electron_eta(cut_of_interest, text_array)
    plot_electron_pt_vs_eta(cut_of_interest, text_array)

    # cut_of_interest, text_array = get_view_at(8, summary_dict)
    # plot_dielectron_inv_mass(cut_of_interest, text_array, False)   

    # cut_of_interest, text_array = get_view_at(9, summary_dict)
    # plot_dimuon_inv_mass(cut_of_interest, text_array, False)

    cut_of_interest, text_array = get_view_at(13, summary_dict, cut_list, events)
    plot_fourL_inv_mass(cut_of_interest, text_array)

def apply_cut_progression(cut_progression, summary_dict, cut_list, events):
    # summary_dict, summary_table = get_summary_of_cuts(events, cut_progression, cut_list)
    # print("Summary of cuts")
    # print(summary_table)

    show_plots_at_each_cut(summary_dict, cut_list, events)

#### Functions to show effect of cuts

In [33]:
def get_cut_name(cut_list, global_index):
    cut_name = cut_list[str(global_index)]['name']
    return cut_name


def make_hist_compare(cuts, var, nbins, xlow, xhigh, xlabel, summary_dict, cut_list, events):
    partial_get_cut_name = functools.partial(get_cut_name, cut_list)

    cut_names = list(map(partial_get_cut_name, cuts))
    print(cut_names)
    found = [item for item in cut_names if 'eleID' in item]
    
    nameExt = found[0] if len(found) > 0 else ''

    unit = 'GeV' if 'eta' not in xlabel else ''
    
    plt.figure(figsize=(8, 8))
    for cut in cuts:
        cut_name = cut_list[str(cut)]['name']
        print(cut_name)
        cut_index = summary_dict['Cut'].index(cut_name)

        mask = summary_dict['Aggregated mask'][cut_index]
        nevents = summary_dict['Events'][cut_index]
        label = f"{summary_dict['Cut'][cut_index]}({nevents})"
    
        plt.hist(ak.flatten(events[var][mask]), bins=nbins, range=(xlow, xhigh), alpha=0.5, label=label)

    plt.yscale('log')
    plt.xlabel(xlabel)
    plt.ylabel(f"Counts / {(xhigh-xlow)/nbins:.2f} {unit}")
    plt.legend(fontsize=13)
    plt.savefig(f"{savepath}/prog_{var}_with {nameExt} cut.png")

def compare_dielectron_mass(cuts, summary_dict, cut_list, events):
    nbins, xlow, xhigh = 60, 70, 110
    var = "B_Z_mass"
    xlabel = 'Dielectron inv mass [GeV]'
    make_hist_compare(cuts, var, nbins, xlow, xhigh, xlabel, summary_dict, cut_list, events)

def compare_dimuon_mass(cuts, summary_dict, cut_list, events):
    nbins, xlow, xhigh = 30, 0, 12
    var = "B_J_mass"
    xlabel = 'Dimuon inv mass [GeV]'
    make_hist_compare(cuts, var, nbins, xlow, xhigh, xlabel, summary_dict, cut_list, events)

def compare_fourL_mass(cuts, summary_dict, cut_list, events):
    nbins, xlow, xhigh = 20, 100, 170
    var = "FourL_mass"
    xlabel = 'FourL inv mass [GeV]'
    make_hist_compare(cuts, var, nbins, xlow, xhigh, xlabel, summary_dict, cut_list, events)

def compare_electron_pt(cuts, summary_dict, cut_list, events):
    nbins, xlow, xhigh = 50, 0, 100
    var = "B_Z_pt1"
    xlabel = "High pT electron [GeV]"
    make_hist_compare(cuts, var, nbins, xlow, xhigh, xlabel, summary_dict, cut_list, events)

    nbins, xlow, xhigh = 25, 0, 50
    var = "B_Z_pt2"
    xlabel = "Low pT electron [GeV]"
    make_hist_compare(cuts, var, nbins, xlow, xhigh, xlabel, summary_dict, cut_list, events)

def compare_muon_pt(cuts, summary_dict, cut_list, events):
    nbins, xlow, xhigh = 40, 0, 45
    var = "B_J_pt1"
    xlabel = "High pT muon [GeV]"
    make_hist_compare(cuts, var, nbins, xlow, xhigh, xlabel, summary_dict, cut_list, events)

    nbins, xlow, xhigh = 30, 0, 20
    var = "B_J_pt2"
    xlabel = "Low pT muon [GeV]"
    make_hist_compare(cuts, var, nbins, xlow, xhigh, xlabel, summary_dict, cut_list, events)

def show_plots_for_single_var(cuts_to_show, summary_dict, cut_list, events):
    compare_dielectron_mass(cuts_to_show, summary_dict, cut_list, events)
    compare_dimuon_mass(cuts_to_show, summary_dict, cut_list, events)
    compare_fourL_mass(cuts_to_show, summary_dict, cut_list, events)
    compare_electron_pt(cuts_to_show, summary_dict, cut_list, events)
    compare_muon_pt(cuts_to_show, summary_dict, cut_list, events)

#### Functions for Gaussian Fitting

In [34]:
def make_fit_plots(summary_dict, cut_of_interest):
    found = [item for item in summary_dict['Cut'] if 'eleID' in item]

    if plot_at >= min(eleID_cuts) and len(found) > 0:
        fileExt = found[0]
    else:
        fileExt = ''

    Z_candidates = ak.flatten(cut_of_interest['Z_mass']).to_numpy()
    Z_mass = ROOT.RooRealVar("Z_mass", "Z_mass", 70, 110, "GeV")
    data = ROOT.RooDataSet.from_numpy({f"Z_mass": Z_candidates}, [Z_mass])

    frame = fit_unbinned_gauss_with_background(data, Z_mass, 20)

    # Draw the frame on the canvas
    canvas = ROOT.TCanvas("canvas", "Z Candidates")
    frame.Draw()
    canvas.Draw()
    canvas.SaveAs(f"{savepath}/fit_Z_gb_with {fileExt} cut.png")


    J_candidates = ak.flatten(cut_of_interest['J_mass']).to_numpy()
    J_mass = ROOT.RooRealVar("J_mass", "J_mass", 2.8, 12, "GeV")
    data = ROOT.RooDataSet.from_numpy({f"J_mass": J_candidates}, [J_mass])

    frame = fit_unbinned_double_gauss(data, J_mass, 20)

    # Draw the frame on the canvas
    canvas = ROOT.TCanvas("canvas", "Y Candidates")
    frame.Draw()
    canvas.Draw()
    canvas.SaveAs(f"{savepath}/fit_J_dg_with {fileExt} cut.png")

## Plotting

In [35]:
def evaluate_a_combination(events, combination, cuts_to_show, cut_list):
    print(f"\nEvaluating combination: {combination}")
    summary_dict, summary_table = get_summary_of_cuts(events, combination, cut_list)
    print(summary_table)
    count = summary_table[summary_table['Cut'] == 'FourL mass']['Events'].values[0]

    # print("\nRemoving dilepton mass cuts")
    # combination_noMass = [x for x in combination if x not in dilepton_mass_cuts]

    # summary_dict, summary_table = get_summary_of_cuts(events, combination_noMass, cut_list)
    # print(summary_table)

    # apply_cut_progression(combination_noMass)
    show_plots_at_each_cut(summary_dict, cut_list, events)

    show_plots_for_single_var(cuts_to_show, summary_dict, cut_list, events)

    return count

In [36]:
# def get_FOM():
#     cuts_to_show = [7, 10, 13]
#     nb = evaluate_a_combination(events, eleID_highPt, cuts_to_show)

#     nEff = evaluate_a_combination(events, eleID_highPt, cuts_to_show)

#     FOM = 1

#     return nb, nEff, FOM

In [37]:
# cuts_to_show = [7, 10, 13]
# nb = evaluate_a_combination(events_data, eleID_highPt, cuts_to_show, cut_list_data)

# cuts_to_show = [7, 11, 13]
# evaluate_a_combination(events_data, eleID_lowPt, cuts_to_show, cut_list_data)

cuts_to_show = [0, 2, 3, 12, 7, 5, 8, 9, 6, 13]
evaluate_a_combination(events_data, eleID_both, cuts_to_show, cut_list_data)


Evaluating combination: [0, 1, 2, 3, 12, 7, 5, 8, 9, 6, 13]
Cut 0: Preselection
Cut 1: UnOrdered pT
Cut 2: Electron Trigger
Cut 3: Soft Muons
Cut 12: eleID either
Cut 7: Detector acceptance
Cut 5: Dilepton Vtx > 1%
Cut 8: J mass
Cut 9: Z mass
Cut 6: FourL Vtx > 1%
Cut 13: FourL mass
                    Cut  Candidates  Events
0          Preselection       74235   52932
1          UnOrdered pT       74195   52900
2      Electron Trigger       14476   11879
3            Soft Muons        4350    3984
4          eleID either        3465    3206
5   Detector acceptance        1785    1660
6     Dilepton Vtx > 1%        1619    1512
7                J mass          89      85
8                Z mass          89      85
9        FourL Vtx > 1%          80      76
10           FourL mass          28      28


/tmp/ipykernel_16513/3712370309.py:23: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure(figsize=(8, 8))


['Preselection', 'Electron Trigger', 'Soft Muons', 'eleID either', 'Detector acceptance', 'Dilepton Vtx > 1%', 'J mass', 'Z mass', 'FourL Vtx > 1%', 'FourL mass']
Preselection
Electron Trigger
Soft Muons
eleID either
Detector acceptance
Dilepton Vtx > 1%
J mass
Z mass
FourL Vtx > 1%
FourL mass
['Preselection', 'Electron Trigger', 'Soft Muons', 'eleID either', 'Detector acceptance', 'Dilepton Vtx > 1%', 'J mass', 'Z mass', 'FourL Vtx > 1%', 'FourL mass']
Preselection
Electron Trigger
Soft Muons
eleID either
Detector acceptance
Dilepton Vtx > 1%
J mass
Z mass
FourL Vtx > 1%
FourL mass
['Preselection', 'Electron Trigger', 'Soft Muons', 'eleID either', 'Detector acceptance', 'Dilepton Vtx > 1%', 'J mass', 'Z mass', 'FourL Vtx > 1%', 'FourL mass']
Preselection
Electron Trigger
Soft Muons
eleID either
Detector acceptance
Dilepton Vtx > 1%
J mass
Z mass
FourL Vtx > 1%
FourL mass
['Preselection', 'Electron Trigger', 'Soft Muons', 'eleID either', 'Detector acceptance', 'Dilepton Vtx > 1%', 'J m

28

#### Plotting at specific cut point

In [38]:
# summary_dict, summary_table = get_summary_of_cuts(events, eleID_highPt, cut_list)
# summary_table
# apply_cut_progression(eleID_highPt)

#### Plots to show effects of cuts

In [39]:
# cuts_to_show = [7, 10, 13]
# show_plots_for_single_var(cuts_to_show, summary_dict)

#### Plots for Gaussian fitting

In [40]:
# summary_dict, summary_table = get_summary_of_cuts(events, eleID_highPt_noMass, cut_list)

# plot_at = 10
# cut_of_interest, text_array = get_view_at(plot_at, summary_dict)
# make_fit_plots(summary_dict, cut_of_interest)

# plot_dimuon_inv_mass(cut_of_interest, text_array, isFitData=True)
# plot_dielectron_inv_mass(cut_of_interest, text_array, isFitData=True)